In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main
! pip install -U openai-whisper

  Cloning https://github.com/huggingface/transformers to c:\users\mh\appdata\local\temp\pip-req-build-xzny_xjp
  Resolved https://github.com/huggingface/transformers to commit a40f1ac602fe900281722254c52ce3773f28eb0e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\mh\AppData\Local\Temp\pip-req-build-xzny_xjp'


In [2]:
! pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [28]:
! pip install optuna


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 8.4 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# Select CUDA device index
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
task = "transcribe"
language = "English"
model_name_or_path = "openai/whisper-base"

In [3]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is enabled and available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Falling back to CPU.")

CUDA is enabled and available!
Using GPU: NVIDIA GeForce RTX 2060 SUPER


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  2 13:39:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060 ...  WDDM  |   00000000:26:00.0  On |                  N/A |
| 38%   45C    P3             28W /  175W |     769MiB /   8192MiB |     37%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Prepare Feature Extractor, Tokenizer and Data

In [5]:
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

### Prepare Data

In [6]:
from datasets import load_dataset, DatasetDict
import os

data_dir = os.path.abspath(os.path.join("..", "..", "data", "processed"))

# Load the dataset
dataset = DatasetDict({
    "train": load_dataset("csv", data_files=os.path.join(data_dir, "train_data.csv")),
    "validation": load_dataset("csv", data_files=os.path.join(data_dir, "validation_data.csv")),
    "test": load_dataset("csv", data_files=os.path.join(data_dir, "test_data.csv"))
})

print(dataset)

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['text', 'path'],
            num_rows: 80
        })
    })
    validation: DatasetDict({
        train: Dataset({
            features: ['text', 'path'],
            num_rows: 10
        })
    })
    test: DatasetDict({
        train: Dataset({
            features: ['text', 'path'],
            num_rows: 10
        })
    })
})


### Batching

In [7]:
# Flatten the nested DatasetDict
dataset = DatasetDict({
    "train": dataset["train"]["train"],  # Extract the inner 'train' dataset
    "validation": dataset["validation"]["train"],  # Extract the inner 'train' dataset
    "test": dataset["test"]["train"]  # Extract the inner 'train' dataset
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'path'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
})


In [8]:
import os

data_dir = os.path.abspath(os.path.join("..", "..", "data", "processed"))

# Function to correct the file paths
def correct_path(example):
    # Convert Windows-style backslashes to forward slashes
    current_path = example["path"].replace("\\", "/")
    
    # Remove '../processed' and replace with '../../data/processed'
    if "../processed" in current_path:
        new_path = current_path.replace("../processed", "../../data/processed")
    else:
        new_path = current_path
    
    example["path"] = new_path
    return example

# Apply the path correction to each split
for split in dataset.keys():
    dataset[split] = dataset[split].map(correct_path)

print(dataset)

# Verify the correction worked
print("Original path:", "../processed\\test\\SP0873-CH00-SE01-RC755.flac")
print("New path:", dataset["test"][0]["path"])
print("New path:", dataset["train"][0]["path"])

DatasetDict({
    train: Dataset({
        features: ['text', 'path'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
})
Original path: ../processed\test\SP0873-CH00-SE01-RC755.flac
New path: ../../data/processed/test/SP0873-CH00-SE01-RC755.flac
New path: ../../data/processed/train/SP0449-CH00-SE00-RC107.flac


In [9]:
from datasets import Audio

# Decode the audio column
dataset = dataset.cast_column("path", Audio(sampling_rate=16000))  # Set the desired sampling rateD

In [10]:
print(dataset["train"][0]["path"])

{'path': '../../data/processed/train/SP0449-CH00-SE00-RC107.flac', 'array': array([-9.46044922e-04, -8.54492188e-04, -9.15527344e-05, ...,
        5.49316406e-04,  6.10351562e-04,  6.10351562e-04]), 'sampling_rate': 16000}


In [11]:
def prepare_dataset(batch):
    # Load and resample audio data
    audio = batch["path"]
    
    # Compute log-Mel input features
    batch["input_features"] = feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

print("Preparing dataset...")
dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset["train"].column_names
)
print("Dataset preparation complete!")

Preparing dataset...
Dataset preparation complete!


## Training & Evaluation

### Define a data collector

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metric

In [14]:
import evaluate
import numpy as np

# Evaluation metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # Replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    # Handle different types of prediction outputs
    if isinstance(pred_ids, list) and isinstance(pred_ids[0], list):
        # Already in the right format
        pass
    elif hasattr(pred_ids, "shape") and len(pred_ids.shape) > 1:
        # Convert numpy arrays to lists if needed
        pred_ids = pred_ids.tolist()
    
    # We don't want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

### Load Pretrained model and apply post-processing

In [15]:
# Load model with quantization
from transformers import BitsAndBytesConfig
from transformers import WhisperForConditionalGeneration
from peft import prepare_model_for_kbit_training

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = WhisperForConditionalGeneration.from_pretrained(
    model_name_or_path, 
    quantization_config=bnb_config,
    device_map="auto"
)

# Set decoding parameters
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

### Hyperparameters Tuning

In [29]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.trainer_utils import HPSearchBackend
import optuna

def model_init():
    # Initialize a fresh model for each trial
    model = WhisperForConditionalGeneration.from_pretrained(
        model_name_or_path, 
        quantization_config=bnb_config,
        device_map="auto"
    )
    
    # Set decoding parameters
    model.config.use_cache = False
    model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
    model.config.suppress_tokens = []
    
    # Prepare model for k-bit training
    model = prepare_model_for_kbit_training(model)
    
    # Apply LoRA
    lora_config = LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none"
    )
    model = get_peft_model(model, lora_config)
    
    return model

# Define the hyperparameter search space
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.3),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1),
    }

# Define the baseline training arguments (hyperparameters will be updated by search)
training_args = Seq2SeqTrainingArguments(
    output_dir="../../output/models/whisper/hp_search",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=1,
    logging_steps=25,
    remove_unused_columns=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_wer",  # WER (lower is better)
    greater_is_better=False,  # For WER, lower is better
    push_to_hub=False,
)

# Initialize trainer with model_init function
trainer = Seq2SeqTrainer(
    args=training_args,
    model_init=model_init,  # Important: use model_init instead of model
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[SavePeftModelCallback],
)

# Run hyperparameter search (n_trials specifies how many combinations to try)
best_trial = trainer.hyperparameter_search(
    direction="minimize",  # Minimize WER
    backend="optuna",
    hp_space=hp_space,
    n_trials=10,  # Adjust based on your computational resources
    compute_objective=lambda metrics: metrics["eval_wer"],
)

print(f"Best hyperparameters: {best_trial.hyperparameters}")

# Train with the best hyperparameters
for param, value in best_trial.hyperparameters.items():
    setattr(trainer.args, param, value)

trainer.train()

e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\training_args.py:1609: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\mh\AppData\Local\Temp\ipykernel_12480\3450653568.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
[I 2025-03-02 14:28:06,077] A new study created in memory with name: no-name-6b2040a9-afb2-4805-a305-c2fab41e445e
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on t

Epoch,Training Loss,Validation Loss,Wer
1,No log,0.483379,17.218543
2,1.457800,0.350180,17.880795
3,0.448600,0.291459,16.556291
4,0.288100,0.278229,15.894040


e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
e:\Singaporean_English_ASR\.venv\lib\site-package

Epoch,Training Loss,Validation Loss,Wer
1,No log,1.557119,74.834437
2,No log,1.488122,66.887417


e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
[I 2025-03-02 14:30:33,703] Trial 1 finished with

...

[I 2025-03-02 14:38:25,622] Trial 9 pruned. 


Best hyperparameters: {'learning_rate': 0.0005288644842478324, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 1, 'num_train_epochs': 5, 'warmup_ratio': 0.1378327739901833, 'weight_decay': 0.009531334178799755}


Epoch,Training Loss,Validation Loss,Wer
1,No log,0.483654,17.880795
2,No log,0.295168,17.218543
3,0.978700,0.255346,13.907285
4,0.978700,0.248491,14.569536
5,0.175900,0.240733,15.231788


e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
e:\Singaporean_English_ASR\.venv\lib\site-package

TrainOutput(global_step=50, training_loss=0.5772707080841064, metrics={'train_runtime': 111.2445, 'train_samples_per_second': 3.596, 'train_steps_per_second': 0.449, 'total_flos': 4.79222562816e+16, 'train_loss': 0.5772707080841064, 'epoch': 5.0})

### LORA

In [16]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],  # For Whisper architecture
    lora_dropout=0.05,
    bias="none"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 73,773,568 || trainable%: 1.5990


### Training Params

In [18]:
from transformers import Seq2SeqTrainingArguments

# Update model configuration before training
model.config.use_cache = False
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
model.config.suppress_tokens = []

# Define a more robust training arguments setup
training_args = Seq2SeqTrainingArguments(
    output_dir="../../output/models/whisper/",
    per_device_train_batch_size=8,  # Reduced from 16 to potentially avoid memory issues
    gradient_accumulation_steps=2,
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=4,  
    predict_with_generate=True,  # Use generation during evaluation
    generation_max_length=128,
    generation_num_beams=1,  # Simple greedy decoding to avoid memory issues
    logging_steps=25,
    remove_unused_columns=False,  # Required for PEFT models
    label_names=["labels"],
    push_to_hub=False,
)


e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\training_args.py:1609: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# Callback to save PEFT adapter weights
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")
        
        # Save the adapter model
        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)
        
        # Remove the PyTorch model file which is not needed for PEFT
        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

In [20]:
from transformers import Seq2SeqTrainer

# Initialize trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,  # Use the full processor instead of just the feature extractor
    callbacks=[SavePeftModelCallback],
)

C:\Users\mh\AppData\Local\Temp\ipykernel_12480\3443089753.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


### Training

In [21]:
# Disable caching during training to avoid memory issues
model.config.use_cache = False

# Start training
print("Starting training...")
trainer.train()
print("Training complete!")

Starting training...


e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Wer
1,No log,1.658005,70.860927
2,No log,1.364233,70.198675
3,No log,0.555613,18.543046


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Training complete!


### Save Model

In [25]:
# Save the final model
model.save_pretrained("../../output/models/whisper/output_models/")

### Evaluation on Test

In [26]:
# Evaluate on test set
print("Evaluating on test set...")
eval_results = trainer.evaluate(dataset["test"])
print(f"Test WER: {eval_results['eval_wer']:.2f}%")

Evaluating on test set...


e:\Singaporean_English_ASR\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Test WER: 21.93%


## Inference

In [ ]:
# Optionally run inference examples
def transcribe_audio(audio_path):
    # Load audio
    from datasets import load_dataset
    audio = load_dataset("audiofolder", data_dir=os.path.dirname(audio_path), 
                          split="train", drop_labels=True)
    audio = audio[0]["audio"]
    
    # Process audio
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    
    # Generate
    model.config.use_cache = True  # Enable cache for inference
    generated_ids = model.generate(input_features=input_features.to("cuda"))
    
    # Decode
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return transcription

In [ ]:
sample_audio = "./data/processed/samples/test_sample.wav"
print(f"Sample transcription: {transcribe_audio(sample_audio)}")